In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
all_drives = pd.read_csv('../data/espn_drives2009-2016.csv')

In [3]:
all_drives.head(5)

,Unnamed: 0,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId
0,0,PIT,0,0,TEN,0,PIT,3,Punt,1:44,2,290910023
1,1,PIT,0,1,TEN,0,TEN,3,Punt,1:52,2,290910023
2,2,PIT,0,2,TEN,0,PIT,5,Punt,3:04,2,290910023
3,3,PIT,0,3,TEN,0,TEN,6,Missed FG,1:35,38,290910023
4,4,PIT,0,4,TEN,0,PIT,3,Punt,1:55,-6,290910023


In [6]:
all_drives['result'].unique()

array(['Punt', 'Missed FG', 'Intercepted Pass', 'Touchdown', 'Fumble',
       'Field Goal', 'End of Half', 'End of Game', 'Downs',
       'End of 1st Half', 'Joshua Cribbs 67 Yard Punt Return',
       'Adrian Peterson 64 Yard Run', 'JAC Timeout', 'NO Timeout',
       'Blocked FG', 'Fumble Touchdown',
       'Desean Jackson 85 Yard Punt Return',
       'Brent Celek 9 Yard Pass From Donovan Mcnabb',
       'Blocked Punt Touchdown', 'Intercepted Pass Touchdown',
       'Julius Jones 62 Yard Run', 'Safety', 'Ryan Grant 3 Yard Run',
       'Blocked Punt', 'Adrian Peterson 27 Yard Run', 'Fumble, Safety',
       'Blocked FG Touchdown',
       'Cadillac Williams 8 Yard Pass From Byron Leftwich', 'BUF Timeout',
       'Frank Gore 79 Yard Run', 'Allen Rossum 67 Yard Punt Return',
       'Will Blackmon 0 Yard Kickoff Return, Tackled In Own End Zone',
       'Percy Harvin 101 Yard Kickoff Return',
       'Ellis Hobbs 0 Yard Kickoff Return, Tackled In Own End Zone',
       'Penalty on Anthony Madis

In [10]:
all_drives['uid'] = all_drives['gameId'].astype(str) +"-"+ all_drives['drive'].astype(str)

In [12]:
all_drives.sample(15)

,Unnamed: 0,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId,uid
45623,5,TB,7,5,CHI,0,CHI,6,Field Goal,2:52,28,400874578,400874578-5
30694,19,STL,3,20,MIN,20,MIN,9,Touchdown,5:23,70,400554217,400554217-20
7629,2,IND,3,2,KC,0,IND,16,Field Goal,7:50,86,301010011,301010011-2
27592,21,HOU,24,22,IND,27,HOU,4,Punt,1:58,24,331103034,331103034-22
6707,17,DEN,31,18,SEA,14,DEN,4,End of Game,1:34,-4,300919007,300919007-18
3179,6,NYJ,10,6,JAX,14,NYJ,6,Punt,2:40,1,291115020,291115020-6
2326,4,DAL,0,4,ATL,7,ATL,3,Punt,1:30,4,291025006,291025006-4
19254,15,MIA,17,16,NYJ,13,MIA,7,Missed FG,2:48,31,320923015,320923015-16
23796,12,NYJ,14,13,SD,10,SD,8,Touchdown,3:49,81,321223020,321223020-13
25834,4,TEN,10,4,NYJ,0,NYJ,10,Field Goal,4:25,63,330929010,330929010-4


In [13]:
# Write a function to count drives in a given dF based on outcome
# Types: TD, FG, Turnover, Punt, End of half/game, Shenanigans
# maybe make columns for each of those types except shenanigans
def find_tds(result_list):
    '''Accepts a list of drive outcomes. 
    Return a binary list of which result in touchdowns'''
    
    is_td = []
    for outcome in [x.lower() for x in result_list]:
        td = 0
        if "touchdown" in outcome:
            if "intercept" in outcome:
                td = 0
            elif "fumble" in outcome:
                td = 0
            elif "punt" in outcome:
                td = 0
            else:
                td = 1
                
        is_td.append(td)
        
    return is_td

def find_fgs(result_list):
    '''Accepts a list of drive outcomes. 
    Return a binary list of which result in field goals'''
    
    is_fg = []
    for outcome in [x.lower() for x in result_list]:
        fg = 0
        if ("field goal" in outcome) or ("fg" in outcome):
            if "block" in outcome:
                fg = 0
            else:
                fg = 1
                
        is_fg.append(fg)
        
    return is_fg

def find_punts(result_list):
    is_punt = []
    for outcome in [x.lower() for x in result_list]:
        punt = 0
        if "punt" in outcome:
            punt = 1
        is_punt.append(punt)
        
    return is_punt
    
def find_turnovers(result_list):
    is_to = []
    for outcome in [x.lower() for x in result_list]:
        to = 0
        if "intercept" in outcome:
            to = 1
        elif "fumble" in outcome:
            to = 1
        elif "downs" in outcome:
            to = 1
        elif "safety" in outcome:
            to = 1
        is_to.append(to)
        
    return is_to
    
def find_endofhalves(result_list):
    is_eoh = []
    for outcome in [x.lower() for x in result_list]:
        eoh = 0
        if "end of half" in outcome:
            eoh = 1
        elif "end of game" in outcome:
            eoh = 1
        is_eoh.append(eoh)
        
    return is_eoh

In [14]:
all_drives['TD'] = find_tds(all_drives['result'].values)

In [15]:
all_drives['FG'] = find_fgs(all_drives['result'].values)
all_drives['punt'] = find_punts(all_drives['result'].values)
all_drives['turnover'] = find_turnovers(all_drives['result'].values)
all_drives['EoH'] = find_endofhalves(all_drives['result'].values)

In [16]:
all_drives.sample(10)

,Unnamed: 0,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId,uid,TD,FG,punt,turnover,EoH
18450,1,TEN,3,1,NE,0,NE,6,Punt,2:14,28,320909010,320909010-1,0,0,1,0,0
32511,19,NYJ,10,20,DEN,24,NYJ,9,Touchdown,4:06,63,400554231,400554231-20,1,0,0,0,0
9310,6,IND,17,6,CIN,0,CIN,8,Field Goal,4:28,55,301114011,301114011-6,0,1,0,0,0
39570,11,SD,16,11,CHI,7,CHI,0,End of Half,0:03,0,400791735,400791735-11,0,0,0,0,1
45543,22,TEN,44,23,GB,25,TEN,9,Field Goal,4:03,44,400874572,400874572-23,0,1,0,0,0
40423,11,KC,14,11,BUF,16,KC,9,Missed FG,1:20,35,400791504,400791504-11,0,1,0,0,0
30552,16,SEA,13,17,STL,3,SEA,14,Touchdown,8:28,69,331229026,331229026-17,1,0,0,0,0
16913,1,MIA,0,1,PHI,0,MIA,3,Touchdown,1:09,15,311211015,311211015-1,1,0,0,0,0
25461,10,NYJ,14,10,BUF,6,NYJ,5,Punt,1:59,16,330922020,330922020-10,0,0,1,0,0
35830,22,SF,35,23,SD,28,SD,14,Touchdown,3:01,80,400554361,400554361-23,1,0,0,0,0


In [17]:
# Summarize a game's drives
subdf = all_drives[ all_drives['gameId'] == 400554231 ]
subdf.groupby(['offense'])[['TD','FG','punt','turnover','EoH']].mean()

,TD,FG,punt,turnover,EoH
offense,,,,,
DEN,0.250000,0.083333,0.666667,0.000000,0.000000
NYJ,0.153846,0.076923,0.538462,0.076923,0.153846
